In [ ]:
import cv2
import numpy as np
import cv2.aruco as aruco
import matplotlib.pyplot as plt

def calculate_deformation(frame, aruco_dict_type, marker_size_mm, reference_size_mm):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco_dict_type)
    parameters = aruco.DetectorParameters_create()
    corners, ids, rejected = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    
    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        marker_size_pixels = np.linalg.norm(corners[0][0][0] - corners[0][0][1])
        pixel_to_mm_ratio = marker_size_mm / marker_size_pixels
        actual_size_mm = reference_size_mm * pixel_to_mm_ratio
        deformation_mm = reference_size_mm - actual_size_mm
        
        cv2.putText(frame, f"Deformation: {deformation_mm:.2f} mm", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        return frame, deformation_mm
    else:
        cv2.putText(frame, "No ArUco markers detected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        return frame, None

def process_video(video_path, aruco_dict_type, marker_size_mm, reference_size_mm):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    deformations = []
    frame_numbers = []
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            processed_frame, deformation = calculate_deformation(frame, aruco_dict_type, marker_size_mm, reference_size_mm)
            
            cv2.imshow('ArUco Marker Detection', processed_frame)
            
            if deformation is not None:
                deformations.append(deformation)
                frame_numbers.append(frame_count)
                print(f"Frame {frame_count}: Deformation = {deformation:.2f} mm")
            
            frame_count += 1
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    return np.array(frame_numbers), np.array(deformations)

def plot_deformation(frame_numbers, deformations):
    plt.figure(figsize=(12, 6))
    plt.plot(frame_numbers, deformations, '-o')
    plt.title('Deformation over Time')
    plt.xlabel('Frame Number')
    plt.ylabel('Deformation (mm)')
    plt.grid(True)
    plt.show()

# Example usage
video_path = "path/to/your/video.mp4"  # Change this to your video file path
aruco_dict_type = aruco.DICT_6X6_250  # Change this based on your ArUco marker type
marker_size_mm = 50  # Change this to the actual size of your ArUco marker in mm
reference_size_mm = 100  # Change this to the reference size of your target in mm

frame_numbers, deformations = process_video(video_path, aruco_dict_type, marker_size_mm, reference_size_mm)
plot_deformation(frame_numbers, deformations)